In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

def add_naive_forecast(df, target_col, months_back, start_date, horizon_months, text):
  
    forecast_col = f"Naive: {months_back}-Month for {text}"
    df[forecast_col] = np.nan

    start = pd.to_datetime(start_date)
    end = start + pd.DateOffset(months=horizon_months - 1)
    reference_date = start - pd.DateOffset(months=months_back)

    if reference_date in df.index:
        naive_value = df[target_col][df.index < start_date].iloc[-1]
        forecast_range = (df.index >= start) & (df.index <= end)
        df.loc[forecast_range, forecast_col] = naive_value

    return df


def getMetrics(df_d, df_r, date1, date2, forecast_type):
    month_name = pd.to_datetime(date1).strftime('%B')
    mask = (df_d.index >= date1) & (df_d.index <= date2)
    y_true = df_d.loc[mask, "Actual Rate"]
    y_pred = df_d.loc[mask, forecast_type]
    forecast_type_split = forecast_type.split(" f")
    if not y_true.isna().all() and not y_pred.isna().all():
        df_r.loc[month_name, f"{forecast_type_split[0]}: MAE"] = mean_absolute_error(y_true, y_pred)
        df_r.loc[month_name, f"{forecast_type_split[0]}: RMSE"] = np.sqrt(mean_squared_error(y_true, y_pred))
    
    return df_r


In [2]:
#1 Month - Prices only
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-01-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("1-Month gas prices only Forecast Error Metrics:")
display(error_metrics)
display(error_details)
print("1-Month Gas prices only \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Jan
Forecast Period: Feb - Feb
Forecast Period: Mar - Mar
Forecast Period: Apr - Apr
Forecast Period: May - May
Forecast Period: Jun - Jun
Forecast Period: Jul - Jul
Forecast Period: Aug - Aug
Forecast Period: Sep - Sep
Forecast Period: Oct - Oct
Forecast Period: Nov - Nov
Forecast Period: Dec - Dec
1-Month gas prices only Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/3919531805.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Jan,45512.286989,45512.286989
1,Forecast Feb - Feb,11358.095935,11358.095935
2,Forecast Mar - Mar,19293.425892,19293.425892
3,Forecast Apr - Apr,9560.812632,9560.812632
4,Forecast May - May,3212.175692,3212.175692
5,Forecast Jun - Jun,8378.420942,8378.420942
6,Forecast Jul - Jul,7841.633034,7841.633034
7,Forecast Aug - Aug,4424.043437,4424.043437
8,Forecast Sep - Sep,21307.125933,21307.125933
9,Forecast Oct - Oct,19154.886664,19154.886664


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,115262.286989,-45512.286989,Forecast Jan - Jan
1,2024-02-01,52375.0,63733.095935,-11358.095935,Forecast Feb - Feb
2,2024-03-01,53950.0,34656.574108,19293.425892,Forecast Mar - Mar
3,2024-04-01,47562.5,57123.312632,-9560.812632,Forecast Apr - Apr
4,2024-05-01,50700.0,47487.824308,3212.175692,Forecast May - May
5,2024-06-01,63312.5,54934.079058,8378.420942,Forecast Jun - Jun
6,2024-07-01,75562.5,67720.866966,7841.633034,Forecast Jul - Jul
7,2024-08-01,76800.0,72375.956563,4424.043437,Forecast Aug - Aug
8,2024-09-01,67000.0,88307.125933,-21307.125933,Forecast Sep - Sep
9,2024-10-01,45312.5,64467.386664,-19154.886664,Forecast Oct - Oct


1-Month Gas prices only 
 MAE : 14253.13 
 RMSE: 18363.19


In [3]:
#1 Month - Gas + Fleet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-01-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("1-Month Gas + Fleet Forecast Error Metrics:")
display(error_metrics)
display(error_details)

print("1-Month Gas + Fleet \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Jan
Forecast Period: Feb - Feb
Forecast Period: Mar - Mar
Forecast Period: Apr - Apr
Forecast Period: May - May
Forecast Period: Jun - Jun
Forecast Period: Jul - Jul
Forecast Period: Aug - Aug
Forecast Period: Sep - Sep
Forecast Period: Oct - Oct
Forecast Period: Nov - Nov
Forecast Period: Dec - Dec
1-Month Gas + Fleet Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/3562061247.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Jan,42526.941279,42526.941279
1,Forecast Feb - Feb,9112.873813,9112.873813
2,Forecast Mar - Mar,19474.099649,19474.099649
3,Forecast Apr - Apr,9096.622758,9096.622758
4,Forecast May - May,4113.737810,4113.737810
5,Forecast Jun - Jun,9396.128273,9396.128273
6,Forecast Jul - Jul,9972.475118,9972.475118
7,Forecast Aug - Aug,3760.497765,3760.497765
8,Forecast Sep - Sep,21238.897250,21238.897250
9,Forecast Oct - Oct,18092.232418,18092.232418


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,112276.941279,-42526.941279,Forecast Jan - Jan
1,2024-02-01,52375.0,61487.873813,-9112.873813,Forecast Feb - Feb
2,2024-03-01,53950.0,34475.900351,19474.099649,Forecast Mar - Mar
3,2024-04-01,47562.5,56659.122758,-9096.622758,Forecast Apr - Apr
4,2024-05-01,50700.0,46586.262190,4113.737810,Forecast May - May
5,2024-06-01,63312.5,53916.371727,9396.128273,Forecast Jun - Jun
6,2024-07-01,75562.5,65590.024882,9972.475118,Forecast Jul - Jul
7,2024-08-01,76800.0,73039.502235,3760.497765,Forecast Aug - Aug
8,2024-09-01,67000.0,88238.897250,-21238.897250,Forecast Sep - Sep
9,2024-10-01,45312.5,63404.732418,-18092.232418,Forecast Oct - Oct


1-Month Gas + Fleet 
 MAE : 14358.81 
 RMSE: 18148.1


In [4]:
#1 Month - Full model
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'LNG Carrier Newbuilding Prices (end month, $m)',
                 'LNG Orderbook (start month, CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-01-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("1-Month full model Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("1-Month full model \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Jan
Forecast Period: Feb - Feb
Forecast Period: Mar - Mar
Forecast Period: Apr - Apr
Forecast Period: May - May
Forecast Period: Jun - Jun
Forecast Period: Jul - Jul
Forecast Period: Aug - Aug
Forecast Period: Sep - Sep
Forecast Period: Oct - Oct
Forecast Period: Nov - Nov
Forecast Period: Dec - Dec
1-Month full model Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/1219925631.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Jan,71636.670413,71636.670413
1,Forecast Feb - Feb,18975.716324,18975.716324
2,Forecast Mar - Mar,15357.606475,15357.606475
3,Forecast Apr - Apr,13056.782207,13056.782207
4,Forecast May - May,5110.648842,5110.648842
5,Forecast Jun - Jun,25157.585329,25157.585329
6,Forecast Jul - Jul,167.377930,167.377930
7,Forecast Aug - Aug,11282.284316,11282.284316
8,Forecast Sep - Sep,27661.448705,27661.448705
9,Forecast Oct - Oct,25483.616336,25483.616336


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,141386.670413,-71636.670413,Forecast Jan - Jan
1,2024-02-01,52375.0,71350.716324,-18975.716324,Forecast Feb - Feb
2,2024-03-01,53950.0,38592.393525,15357.606475,Forecast Mar - Mar
3,2024-04-01,47562.5,60619.282207,-13056.782207,Forecast Apr - Apr
4,2024-05-01,50700.0,45589.351158,5110.648842,Forecast May - May
5,2024-06-01,63312.5,38154.914671,25157.585329,Forecast Jun - Jun
6,2024-07-01,75562.5,75395.122070,167.377930,Forecast Jul - Jul
7,2024-08-01,76800.0,65517.715684,11282.284316,Forecast Aug - Aug
8,2024-09-01,67000.0,94661.448705,-27661.448705,Forecast Sep - Sep
9,2024-10-01,45312.5,70796.116336,-25483.616336,Forecast Oct - Oct


1-Month full model 
 MAE : 19309.57 
 RMSE: 26368.44


In [5]:
#3 Month - Prices only
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-03-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("3-Month gas prices only Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("3-Month Gas prices only \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Mar
Forecast Period: Feb - Apr
Forecast Period: Mar - May
Forecast Period: Apr - Jun
Forecast Period: May - Jul
Forecast Period: Jun - Aug
Forecast Period: Jul - Sep
Forecast Period: Aug - Oct
Forecast Period: Sep - Nov
Forecast Period: Oct - Dec
3-Month gas prices only Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/2899630931.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Mar,52143.319484,52373.359668
1,Forecast Feb - Apr,11437.367881,11758.321286
2,Forecast Mar - May,19776.664563,20021.316924
3,Forecast Apr - Jun,6472.704253,7804.668517
4,Forecast May - Jul,15870.837984,18873.501387
5,Forecast Jun - Aug,15335.368709,16110.965235
6,Forecast Jul - Sep,6793.876157,6843.561503
7,Forecast Aug - Oct,15531.672811,20010.204772
8,Forecast Sep - Nov,50824.672584,55956.438423
9,Forecast Oct - Dec,31346.848238,32543.954709


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,115262.286989,-45512.286989,Forecast Jan - Mar
1,2024-02-01,52375.0,109590.710860,-57215.710860,Forecast Jan - Mar
2,2024-03-01,53950.0,107651.960602,-53701.960602,Forecast Jan - Mar
3,2024-02-01,52375.0,63733.095935,-11358.095935,Forecast Feb - Apr
4,2024-03-01,53950.0,62085.987529,-8135.987529,Forecast Feb - Apr
5,2024-04-01,47562.5,62380.520180,-14818.020180,Forecast Feb - Apr
6,2024-03-01,53950.0,34656.574108,19293.425892,Forecast Mar - May
7,2024-04-01,47562.5,31342.882382,16219.617618,Forecast Mar - May
8,2024-05-01,50700.0,26883.049820,23816.950180,Forecast Mar - May
9,2024-04-01,47562.5,57123.312632,-9560.812632,Forecast Apr - Jun


3-Month Gas prices only 
 MAE : 22553.33 
 RMSE: 29319.34


In [6]:
#3 Month - Gas + Fleet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-03-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("3-Month Gas + Fleet Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("3-Month Gas + Fleet \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Mar
Forecast Period: Feb - Apr
Forecast Period: Mar - May
Forecast Period: Apr - Jun
Forecast Period: May - Jul
Forecast Period: Jun - Aug
Forecast Period: Jul - Sep
Forecast Period: Aug - Oct
Forecast Period: Sep - Nov
Forecast Period: Oct - Dec
3-Month Gas + Fleet Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/3483504441.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Mar,41837.734708,41992.863972
1,Forecast Feb - Apr,8127.636900,8724.807574
2,Forecast Mar - May,21630.685207,22002.628902
3,Forecast Apr - Jun,6585.023783,7191.915045
4,Forecast May - Jul,17222.027203,20148.373994
5,Forecast Jun - Aug,16134.889984,16846.509351
6,Forecast Jul - Sep,7588.182776,9013.201493
7,Forecast Aug - Oct,14963.732532,19195.037494
8,Forecast Sep - Nov,51394.868962,56728.070191
9,Forecast Oct - Dec,29270.046661,30347.190796


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,112276.941279,-42526.941279,Forecast Jan - Mar
1,2024-02-01,52375.0,98244.272910,-45869.272910,Forecast Jan - Mar
2,2024-03-01,53950.0,91066.989936,-37116.989936,Forecast Jan - Mar
3,2024-02-01,52375.0,61487.873813,-9112.873813,Forecast Feb - Apr
4,2024-03-01,53950.0,57794.548867,-3844.548867,Forecast Feb - Apr
5,2024-04-01,47562.5,58987.988018,-11425.488018,Forecast Feb - Apr
6,2024-03-01,53950.0,34475.900351,19474.099649,Forecast Mar - May
7,2024-04-01,47562.5,29420.310623,18142.189377,Forecast Mar - May
8,2024-05-01,50700.0,23424.233406,27275.766594,Forecast Mar - May
9,2024-04-01,47562.5,56659.122758,-9096.622758,Forecast Apr - Jun


3-Month Gas + Fleet 
 MAE : 21475.48 
 RMSE: 27665.57


In [7]:
#3 Month - Full model
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'LNG Carrier Newbuilding Prices (end month, $m)',
                 'LNG Orderbook (start month, CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-03-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("3-Month full model Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("3-Month full model \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Mar
Forecast Period: Feb - Apr
Forecast Period: Mar - May
Forecast Period: Apr - Jun
Forecast Period: May - Jul
Forecast Period: Jun - Aug
Forecast Period: Jul - Sep
Forecast Period: Aug - Oct
Forecast Period: Sep - Nov
Forecast Period: Oct - Dec
3-Month full model Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/3960570783.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Mar,66103.923869,66292.598116
1,Forecast Feb - Apr,17288.161159,17420.856332
2,Forecast Mar - May,18942.709881,19334.469366
3,Forecast Apr - Jun,9173.808963,10749.753195
4,Forecast May - Jul,20419.250531,23764.642216
5,Forecast Jun - Aug,32310.996000,32717.031355
6,Forecast Jul - Sep,3840.569379,4819.571314
7,Forecast Aug - Oct,13344.261480,15585.669635
8,Forecast Sep - Nov,55035.586326,59217.768036
9,Forecast Oct - Dec,33980.131071,34522.426163


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,141386.670413,-71636.670413,Forecast Jan - Mar
1,2024-02-01,52375.0,119521.915084,-67146.915084,Forecast Jan - Mar
2,2024-03-01,53950.0,113478.186112,-59528.186112,Forecast Jan - Mar
3,2024-02-01,52375.0,71350.716324,-18975.716324,Forecast Feb - Apr
4,2024-03-01,53950.0,68209.733300,-14259.733300,Forecast Feb - Apr
5,2024-04-01,47562.5,66191.533852,-18629.033852,Forecast Feb - Apr
6,2024-03-01,53950.0,38592.393525,15357.606475,Forecast Mar - May
7,2024-04-01,47562.5,30412.408876,17150.091124,Forecast Mar - May
8,2024-05-01,50700.0,26379.567957,24320.432043,Forecast Mar - May
9,2024-04-01,47562.5,60619.282207,-13056.782207,Forecast Apr - Jun


3-Month full model 
 MAE : 27043.94 
 RMSE: 34332.81


In [8]:
#6 Month - Prices only
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-06-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("6-Month gas prices only Forecast Error Metrics:")
display(error_metrics)
display(error_details)

print("6-Month Gas prices only \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Jun
Forecast Period: Feb - Jul
Forecast Period: Mar - Aug
Forecast Period: Apr - Sep
Forecast Period: May - Oct
Forecast Period: Jun - Nov
Forecast Period: Jul - Dec
6-Month gas prices only Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/1602896086.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Jun,57645.661573,58047.292177
1,Forecast Feb - Jul,9574.253344,10406.715477
2,Forecast Mar - Aug,32874.066863,35756.553537
3,Forecast Apr - Sep,6955.446757,7821.376737
4,Forecast May - Oct,16539.225864,19320.612944
5,Forecast Jun - Nov,17837.171790,20405.333129
6,Forecast Jul - Dec,26686.923761,34350.213355


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,115262.286989,-45512.286989,Forecast Jan - Jun
1,2024-02-01,52375.0,109590.710860,-57215.710860,Forecast Jan - Jun
2,2024-03-01,53950.0,107651.960602,-53701.960602,Forecast Jan - Jun
3,2024-04-01,47562.5,111872.659015,-64310.159015,Forecast Jan - Jun
4,2024-05-01,50700.0,116657.011443,-65957.011443,Forecast Jan - Jun
5,2024-06-01,63312.5,122489.340528,-59176.840528,Forecast Jan - Jun
6,2024-02-01,52375.0,63733.095935,-11358.095935,Forecast Feb - Jul
7,2024-03-01,53950.0,62085.987529,-8135.987529,Forecast Feb - Jul
8,2024-04-01,47562.5,62380.520180,-14818.020180,Forecast Feb - Jul
9,2024-05-01,50700.0,63893.664680,-13193.664680,Forecast Feb - Jul


6-Month Gas prices only 
 MAE : 24016.11 
 RMSE: 31138.16


In [9]:
#6 Month - Gas + Fleet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-06-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("6-Month Gas + Fleet Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("6-Month Gas + Fleet \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Jun
Forecast Period: Feb - Jul


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/1509273263.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

Forecast Period: Mar - Aug
Forecast Period: Apr - Sep
Forecast Period: May - Oct
Forecast Period: Jun - Nov
Forecast Period: Jul - Dec
6-Month Gas + Fleet Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/F

,Forecast Type,MAE,RMSE
0,Forecast Jan - Jun,40955.566107,41165.611708
1,Forecast Feb - Jul,7623.422071,8575.529381
2,Forecast Mar - Aug,36272.629359,39534.775684
3,Forecast Apr - Sep,7416.644916,8757.619849
4,Forecast May - Oct,17449.645621,20543.422340
5,Forecast Jun - Nov,18368.404068,20888.733012
6,Forecast Jul - Dec,22784.225794,28740.036359


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,112276.941279,-42526.941279,Forecast Jan - Jun
1,2024-02-01,52375.0,98244.272910,-45869.272910,Forecast Jan - Jun
2,2024-03-01,53950.0,91066.989936,-37116.989936,Forecast Jan - Jun
3,2024-04-01,47562.5,91415.845502,-43853.345502,Forecast Jan - Jun
4,2024-05-01,50700.0,93264.067123,-42564.067123,Forecast Jan - Jun
5,2024-06-01,63312.5,97115.279890,-33802.779890,Forecast Jan - Jun
6,2024-02-01,52375.0,61487.873813,-9112.873813,Forecast Feb - Jul
7,2024-03-01,53950.0,57794.548867,-3844.548867,Forecast Feb - Jul
8,2024-04-01,47562.5,58987.988018,-11425.488018,Forecast Feb - Jul
9,2024-05-01,50700.0,60404.157424,-9704.157424,Forecast Feb - Jul


6-Month Gas + Fleet 
 MAE : 21552.93 
 RMSE: 26971.42


In [19]:
#6 Month - Full model
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'LNG Carrier Newbuilding Prices (end month, $m)',
                 'LNG Orderbook (start month, CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-06-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("6-Month full model Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("6-Month full model \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Jun
Forecast Period: Feb - Jul
Forecast Period: Mar - Aug
Forecast Period: Apr - Sep
Forecast Period: May - Oct
Forecast Period: Jun - Nov
Forecast Period: Jul - Dec
6-Month full model Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/3047359302.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Jun,63849.142798,64099.611058
1,Forecast Feb - Jul,13155.764340,14396.427254
2,Forecast Mar - Aug,34063.547476,37699.668183
3,Forecast Apr - Sep,8167.718336,9722.705036
4,Forecast May - Oct,21457.518530,24965.604476
5,Forecast Jun - Nov,23701.280649,26278.709746
6,Forecast Jul - Dec,24659.194233,33290.530937


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,141386.670413,-71636.670413,Forecast Jan - Jun
1,2024-02-01,52375.0,119521.915084,-67146.915084,Forecast Jan - Jun
2,2024-03-01,53950.0,113478.186112,-59528.186112,Forecast Jan - Jun
3,2024-04-01,47562.5,113759.094149,-66196.594149,Forecast Jan - Jun
4,2024-05-01,50700.0,115240.536627,-64540.536627,Forecast Jan - Jun
5,2024-06-01,63312.5,117358.454401,-54045.954401,Forecast Jan - Jun
6,2024-02-01,52375.0,71350.716324,-18975.716324,Forecast Feb - Jul
7,2024-03-01,53950.0,68209.733300,-14259.733300,Forecast Feb - Jul
8,2024-04-01,47562.5,66191.533852,-18629.033852,Forecast Feb - Jul
9,2024-05-01,50700.0,66666.501227,-15966.501227,Forecast Feb - Jul


6-Month full model 
 MAE : 27007.74 
 RMSE: 34338.49


In [23]:
#12 Month - Prices only
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-12-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("12-Month gas prices only Forecast Error Metrics:")
display(error_metrics)
display(error_details)

print("12-Month Gas prices only \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Dec
12-Month gas prices only Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/457928167.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])

,Forecast Type,MAE,RMSE
0,Forecast Jan - Dec,76004.534913,82077.206256


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,115262.286989,-45512.286989,Forecast Jan - Dec
1,2024-02-01,52375.0,109590.710860,-57215.710860,Forecast Jan - Dec
2,2024-03-01,53950.0,107651.960602,-53701.960602,Forecast Jan - Dec
3,2024-04-01,47562.5,111872.659015,-64310.159015,Forecast Jan - Dec
4,2024-05-01,50700.0,116657.011443,-65957.011443,Forecast Jan - Dec
5,2024-06-01,63312.5,122489.340528,-59176.840528,Forecast Jan - Dec
6,2024-07-01,75562.5,128659.902425,-53097.402425,Forecast Jan - Dec
7,2024-08-01,76800.0,135347.759374,-58547.759374,Forecast Jan - Dec
8,2024-09-01,67000.0,142436.179237,-75436.179237,Forecast Jan - Dec
9,2024-10-01,45312.5,149944.396475,-104631.896475,Forecast Jan - Dec


12-Month Gas prices only 
 MAE : 76004.53 
 RMSE: 82077.21


In [25]:
#12 Month - Gas + Fleet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-12-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("12-Month Gas + Fleet Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("12-Month Gas + Fleet \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Dec
12-Month Gas + Fleet Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/4187602595.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"]

,Forecast Type,MAE,RMSE
0,Forecast Jan - Dec,50992.137955,56335.428704


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,112276.941279,-42526.941279,Forecast Jan - Dec
1,2024-02-01,52375.0,98244.272910,-45869.272910,Forecast Jan - Dec
2,2024-03-01,53950.0,91066.989936,-37116.989936,Forecast Jan - Dec
3,2024-04-01,47562.5,91415.845502,-43853.345502,Forecast Jan - Dec
4,2024-05-01,50700.0,93264.067123,-42564.067123,Forecast Jan - Dec
5,2024-06-01,63312.5,97115.279890,-33802.779890,Forecast Jan - Dec
6,2024-07-01,75562.5,101190.261604,-25627.761604,Forecast Jan - Dec
7,2024-08-01,76800.0,105801.676701,-29001.676701,Forecast Jan - Dec
8,2024-09-01,67000.0,110572.482990,-43572.482990,Forecast Jan - Dec
9,2024-10-01,45312.5,115617.343938,-70304.843938,Forecast Jan - Dec


12-Month Gas + Fleet 
 MAE : 50992.14 
 RMSE: 56335.43


In [24]:
#12 Month - Full model
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

file_path = "../cleandata.csv"
df_clean = pd.read_csv(file_path)

df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b-%Y')
df_clean = df_clean.sort_values(by='Date').set_index('Date')
forecast_data = ['LNG 174K CBM (2-stroke dual fuel) Spot Rate (avg., $/day)',
                 'LNG Shipping Capacity (CBM)',
                 'LNG Carrier Newbuilding Prices (end month, $m)',
                 'LNG Orderbook (start month, CBM)',
                 'Global price of Natural Gas, Asia (start month, MMBTU)',
                 'Global price of Natural Gas, EU (start month, MMBTU)',
                 'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
                 'Spread\nEU-US',
                 'Spread\nASIA-US',
                 'Spread\nASIA-EU']
plotskip =  ['LNG Shipping Capacity (CBM)',
             'LNG Carrier Newbuilding Prices (end month, $m)',
             'LNG Orderbook (start month, CBM)',
             'Global price of Natural Gas, Asia (start month, MMBTU)',
             'Global price of Natural Gas, EU (start month, MMBTU)',
             'Global price of Natural Gas, US Henry Hub (start month, MMBTU)',
             'Spread\nEU-US',
             'Spread\nASIA-US',
             'Spread\nASIA-EU']

target = forecast_data[0]

forecast_start_date = pd.to_datetime("2024-01-01")
forecast_end_date = pd.to_datetime("2024-12-01")
df_forecast = df_clean[forecast_data].copy()
error_metrics = pd.DataFrame(columns=["Forecast Type", "MAE", "RMSE"])
error_details = pd.DataFrame()

while forecast_end_date <= pd.to_datetime("2024-12-01"):
    print(f"Forecast Period: {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}")
    
    df_temp = df_clean[forecast_data].copy()
    df_train = df_temp[df_temp.index < forecast_start_date]
    df_train = np.log(df_train).diff().dropna()
    
    # Generate VAR forecast
    forecast_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq="MS")
    forecast_steps = len(forecast_range)
    model = VAR(df_train)
    selected_lag = model.select_order(maxlags=2).aic
    if selected_lag is None or selected_lag == 0:
        selected_lag = 1
    results = model.fit()
    forecast = results.forecast(df_train.values, steps=forecast_steps)
    
    # Recover the forecast values
    last_log_value = np.log(df_forecast.loc[df_train.index[-1], target])
    df_temp_forecast = pd.DataFrame(forecast, index=forecast_range, columns=df_train.columns)
    log_forecast_cumsum = df_temp_forecast[target].cumsum() + last_log_value  
    recovered_forecast = np.exp(log_forecast_cumsum) 
    forecast_label = f"Forecast {forecast_start_date.strftime('%b')} - {forecast_end_date.strftime('%b')}"
    df_forecast.loc[forecast_range, forecast_label] = recovered_forecast
    
    # Calculate error metrics
    mask = (df_forecast.index >= forecast_start_date) & (df_forecast.index <= forecast_end_date)
    y_true = df_forecast.loc[mask, target]
    y_pred = df_forecast.loc[mask, forecast_label]
    if not y_true.isna().all() and not y_pred.isna().all():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])])
        details_df = pd.DataFrame({"Date": y_true.index, "Actual": y_true.values, "Forecast": y_pred.values, "Error": (y_true - y_pred).values, "Forecast Type": forecast_label})
        error_details = pd.concat([error_details, details_df])
    
    # Move to the next 3-month period
    forecast_start_date += pd.DateOffset(months=1)
    forecast_end_date += pd.DateOffset(months=1)

# Reset index for cleaner output
error_metrics.reset_index(drop=True, inplace=True)
error_details.reset_index(drop=True, inplace=True)

# Display error metrics
print("12-Month full model Forecast Error Metrics:")
display(error_metrics)
display(error_details)


print("12-Month full model \n",
      "MAE :", round(mean_absolute_error(error_details["Actual"],error_details["Forecast"]),2),"\n",
      "RMSE:", round(np.sqrt(mean_squared_error(error_details['Actual'],error_details["Forecast"])),2))


Forecast Period: Jan - Dec
12-Month full model Forecast Error Metrics:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_37097/652947274.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  error_metrics = pd.concat([error_metrics, pd.DataFrame([[forecast_label, mae, rmse]], columns=["Forecast Type", "MAE", "RMSE"])

,Forecast Type,MAE,RMSE
0,Forecast Jan - Dec,72507.723669,76367.806085


,Date,Actual,Forecast,Error,Forecast Type
0,2024-01-01,69750.0,141386.670413,-71636.670413,Forecast Jan - Dec
1,2024-02-01,52375.0,119521.915084,-67146.915084,Forecast Jan - Dec
2,2024-03-01,53950.0,113478.186112,-59528.186112,Forecast Jan - Dec
3,2024-04-01,47562.5,113759.094149,-66196.594149,Forecast Jan - Dec
4,2024-05-01,50700.0,115240.536627,-64540.536627,Forecast Jan - Dec
5,2024-06-01,63312.5,117358.454401,-54045.954401,Forecast Jan - Dec
6,2024-07-01,75562.5,121121.715104,-45559.215104,Forecast Jan - Dec
7,2024-08-01,76800.0,125056.891794,-48256.891794,Forecast Jan - Dec
8,2024-09-01,67000.0,130204.509633,-63204.509633,Forecast Jan - Dec
9,2024-10-01,45312.5,135494.241558,-90181.741558,Forecast Jan - Dec


12-Month full model 
 MAE : 72507.72 
 RMSE: 76367.81
